In [1]:
from datasets import load_dataset
import os
from abc import ABC, abstractmethod
import pandas as pd
import json
from typing import List, Dict, Any
from collections import Counter
import numpy as np
import os
import json
import random
from abc import ABC, abstractmethod
from typing import List, Dict, Any
from collections import Counter
import pandas as pd
import numpy as np
from datasets import load_from_disk

In [4]:
Create directory for HF datasets
os.makedirs('/kaggle/working/hf_datasets', exist_ok=True)

print("Downloading GSM-Symbolic...")
wp = load_dataset("apple/GSM-Symbolic")
wp.save_to_disk('/kaggle/working/hf_datasets/GSM-Symbolic')

print("\nDownloading LitBench...")
wp = load_dataset("SAA-Lab/LitBench-Rationales")
wp.save_to_disk('/kaggle/working/hf_datasets/LitBench-Rationales')

print("\nDownloading qmsum-cleaned...")
wp = load_dataset("pszemraj/qmsum-cleaned")
wp.save_to_disk('/kaggle/working/hf_datasets/qmsum-cleaned')

print("\n5/7 HumanEval+...")
he_plus = load_dataset("evalplus/humanevalplus")
he_plus.save_to_disk('/kaggle/working/hf_datasets/humanevalplus')

# 6. SciQ (Basic Science)
print("\n6/7 SciQ...")
sciq = load_dataset("allenai/sciq")
sciq.save_to_disk('/kaggle/working/hf_datasets/sciq')

7. LiveIdeaBench (Creative Ideation / Divergent Thinking)
print("\n7/7 LiveIdeaBench...")
idea_bench = load_dataset("6cf/liveideabench-v2")
idea_bench.save_to_disk('/kaggle/working/hf_datasets/liveideabench')

In [16]:
# ==================== CONFIGURATION ====================

# Base path where HF datasets are saved
HF_DATASETS_PATH = "/kaggle/working/hf_datasets"

# Output path
OUTPUT_PATH = "/kaggle/working/stress_test_dataset.jsonl"

# Sample sizes per domain (keep small for stress test)
SAMPLES_PER_DOMAIN = 100

# Domains to include
DOMAINS = [
    "math",
    "coding",
    "science",
    "summarization", 
    "logic",
]

# ==================== UTILITIES ====================

def to_json_serializable(obj: Any) -> Any:
    """Convert numpy/pandas objects to JSON-serializable Python types"""
    if isinstance(obj, (np.ndarray, pd.Series)):
        return obj.tolist()
    elif isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    elif isinstance(obj, dict):
        return {k: to_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [to_json_serializable(item) for item in obj]
    elif pd.isna(obj):
        return None
    else:
        return obj


# ==================== BASE ADAPTER ====================

class DatasetAdapter(ABC):
    """Base adapter for all datasets"""
    
    def __init__(self, domain: str, source_name: str):
        self.domain = domain
        self.source_name = source_name
    
    @abstractmethod
    def load(self) -> pd.DataFrame:
        """Load raw data"""
        pass
    
    @abstractmethod
    def transform(self, df: pd.DataFrame) -> List[Dict]:
        """Transform to unified schema"""
        pass
    
    def validate(self, data: List[Dict]) -> List[Dict]:
        """Validate required fields and quality"""
        validated = []
        for item in data:
            # Check required fields
            if not all(k in item for k in ['domain', 'prompt', 'answer']):
                continue
            
            # Check prompt length (>5 words)
            if len(item['prompt'].split()) < 5:
                continue
            
            # Check answer not empty
            if len(str(item['answer']).strip()) == 0:
                continue
            
            validated.append(item)
        
        return validated
    
    def process(self, max_samples: int = None) -> List[Dict]:
        """Full pipeline: load → transform → validate → sample"""
        print(f"\n{'='*50}")
        print(f"Processing {self.source_name}...")
        print(f"{'='*50}")
        
        try:
            # Load
            print("  [1/4] Loading raw data...")
            df = self.load()
            print(f"        Loaded {len(df)} rows")
            
            # Transform
            print("  [2/4] Transforming to unified schema...")
            transformed = self.transform(df)
            print(f"        Transformed {len(transformed)} samples")
            
            # Validate
            print("  [3/4] Validating...")
            validated = self.validate(transformed)
            print(f"        Validated {len(validated)} samples")
            
            # Sample if needed
            if max_samples and len(validated) > max_samples:
                print(f"  [4/4] Sampling {max_samples} from {len(validated)}...")
                validated = random.sample(validated, max_samples)
            
            return validated
            
        except Exception as e:
            print(f"  ERROR: {e}")
            return []


# ==================== CONCRETE ADAPTERS ====================

class GSMSymbolicAdapter(DatasetAdapter):
    """Adapter for GSM-Symbolic math dataset"""
    
    def __init__(self):
        super().__init__(domain="math", source_name="GSM-Symbolic")
        self.path = f"{HF_DATASETS_PATH}/GSM-Symbolic"
    
    def load(self) -> pd.DataFrame:
        dataset = load_from_disk(self.path)
        # Use test split
        if 'test' in dataset:
            return dataset['test'].to_pandas()
        return dataset.to_pandas()
    
    def transform(self, df: pd.DataFrame) -> List[Dict]:
        data = []
        for _, row in df.iterrows():
            # GSM-Symbolic typically has 'question' and 'answer' columns
            question = row.get('question', row.get('problem', ''))
            answer = row.get('answer', row.get('solution', ''))
            
            if question and answer:
                data.append({
                    "domain": self.domain,
                    "prompt": str(question),
                    "answer": str(answer),
                    "metadata": {"source": self.source_name}
                })
        return data


class HumanEvalPlusAdapter(DatasetAdapter):
    """Adapter for HumanEvalPlus coding dataset"""
    
    def __init__(self):
        super().__init__(domain="coding", source_name="HumanEvalPlus")
        self.path = f"{HF_DATASETS_PATH}/humanevalplus"
    
    def load(self) -> pd.DataFrame:
        dataset = load_from_disk(self.path)
        if 'test' in dataset:
            return dataset['test'].to_pandas()
        return dataset.to_pandas()
    
    def transform(self, df: pd.DataFrame) -> List[Dict]:
        data = []
        for _, row in df.iterrows():
            # HumanEval has 'prompt' (docstring) and 'canonical_solution'
            prompt = row.get('prompt', '')
            solution = row.get('canonical_solution', row.get('solution', ''))
            
            # Create task prompt
            task_prompt = f"Write a Python function to solve the following problem:\n\n{prompt}"
            
            if prompt and solution:
                data.append({
                    "domain": self.domain,
                    "prompt": task_prompt,
                    "answer": str(solution),
                    "metadata": {
                        "source": self.source_name,
                        "entry_point": row.get('entry_point', ''),
                    }
                })
        return data


class SciQAdapter(DatasetAdapter):
    """Adapter for SciQ science dataset"""
    
    def __init__(self):
        super().__init__(domain="science", source_name="SciQ")
        self.path = f"{HF_DATASETS_PATH}/sciq"
    
    def load(self) -> pd.DataFrame:
        dataset = load_from_disk(self.path)
        # Merge all splits
        all_data = []
        for split in dataset.keys():
            all_data.append(dataset[split].to_pandas())
        return pd.concat(all_data, ignore_index=True)
    
    def transform(self, df: pd.DataFrame) -> List[Dict]:
        data = []
        for _, row in df.iterrows():
            question = row.get('question', '')
            answer = row.get('correct_answer', '')
            support = row.get('support', '')
            
            # Include supporting context if available
            if support:
                full_prompt = f"Based on the following context, answer the question.\n\nContext: {support}\n\nQuestion: {question}"
            else:
                full_prompt = question
            
            if question and answer:
                data.append({
                    "domain": self.domain,
                    "prompt": full_prompt,
                    "answer": str(answer),
                    "metadata": {"source": self.source_name}
                })
        return data


class QMSumAdapter(DatasetAdapter):
    """Adapter for QMSum summarization dataset"""
    
    def __init__(self):
        super().__init__(domain="summarization", source_name="QMSum")
        self.path = f"{HF_DATASETS_PATH}/qmsum-cleaned"
    
    def load(self) -> pd.DataFrame:
        dataset = load_from_disk(self.path)
        # Prefer test split for validation
        if 'test' in dataset:
            return dataset['test'].to_pandas()
        elif 'validation' in dataset:
            return dataset['validation'].to_pandas()
        return dataset.to_pandas()
    
    def transform(self, df: pd.DataFrame) -> List[Dict]:
        data = []
        for _, row in df.iterrows():
            # QMSum has meeting transcripts and summaries
            source = row.get('source', row.get('input', row.get('transcript', '')))
            summary = row.get('target', row.get('summary', row.get('output', '')))
            
            if source and summary:
                # Truncate long sources for reasonable prompt length
                if len(source) > 3000:
                    source = source[:3000] + "..."
                
                prompt = f"Summarize the following meeting transcript:\n\n{source}"
                
                data.append({
                    "domain": self.domain,
                    "prompt": prompt,
                    "answer": str(summary),
                    "metadata": {"source": self.source_name}
                })
        return data


class LitBenchAdapter(DatasetAdapter):
    """Adapter for LitBench-Rationales logic/reasoning dataset"""
    
    def __init__(self):
        super().__init__(domain="logic", source_name="LitBench-Rationales")
        self.path = f"{HF_DATASETS_PATH}/LitBench-Rationales"
    
    def load(self) -> pd.DataFrame:
        dataset = load_from_disk(self.path)
        if 'train' in dataset:
            return dataset['train'].to_pandas()
        return dataset.to_pandas()
    
    def transform(self, df: pd.DataFrame) -> List[Dict]:
        data = []
        for _, row in df.iterrows():
            # LitBench has questions and rationales
            question = row.get('question', row.get('prompt', row.get('input', '')))
            answer = row.get('answer', row.get('rationale', row.get('output', '')))
            
            if question and answer:
                data.append({
                    "domain": self.domain,
                    "prompt": str(question),
                    "answer": str(answer),
                    "metadata": {"source": self.source_name}
                })
        return data


# ==================== ADVERSARIAL EXAMPLES ====================

def generate_adversarial_examples() -> List[Dict]:
    """Generate edge case examples for stress testing"""
    
    adversarial = []
    
    # 1. Very short questions
    short_questions = [
        {"prompt": "What is 2+2?", "answer": "4", "domain": "math"},
        {"prompt": "Is water wet?", "answer": "Yes", "domain": "science"},
        {"prompt": "What color is the sky?", "answer": "Blue", "domain": "logic"},
        {"prompt": "Write hello world in Python.", "answer": "print('hello world')", "domain": "coding"},
        {"prompt": "Summarize: The cat sat.", "answer": "A cat sat.", "domain": "summarization"},
    ]
    
    # 2. Multi-part questions  
    multipart_questions = [
        {
            "prompt": "First, calculate 15 + 27. Then, multiply the result by 3. Finally, subtract 10 from that. What is the final answer?",
            "answer": "The final answer is 116. (15+27=42, 42×3=126, 126-10=116)",
            "domain": "math"
        },
        {
            "prompt": "What are the three states of matter? For each state, give one example.",
            "answer": "Solid (ice), Liquid (water), Gas (steam)",
            "domain": "science"
        },
        {
            "prompt": "Write a function that: 1) Takes a list of numbers 2) Filters out negatives 3) Returns the sum",
            "answer": "def sum_positives(nums): return sum(x for x in nums if x >= 0)",
            "domain": "coding"
        },
    ]
    
    # 3. Potentially ambiguous questions
    ambiguous_questions = [
        {
            "prompt": "What is the answer?",
            "answer": "42 (The answer to life, the universe, and everything)",
            "domain": "logic"
        },
        {
            "prompt": "Solve the problem.",
            "answer": "I need more context to solve a specific problem.",
            "domain": "math"
        },
        {
            "prompt": "Tell me about it.",
            "answer": "Please specify what you would like to know about.",
            "domain": "logic"
        },
    ]
    
    # Combine all
    for q in short_questions + multipart_questions + ambiguous_questions:
        q["metadata"] = {"source": "adversarial"}
        adversarial.append(q)
    
    print(f"\n{'='*50}")
    print(f"Generated {len(adversarial)} adversarial examples")
    print(f"{'='*50}")
    
    return adversarial


# ==================== DATASET MIXER ====================

class StressTestMixer:
    """Combines all adapters into stress test dataset"""
    
    def __init__(self):
        self.adapters = [
            GSMSymbolicAdapter(),
            HumanEvalPlusAdapter(),
            SciQAdapter(),
            QMSumAdapter(),
            LitBenchAdapter(),
        ]
    
    def create_stress_test_dataset(self, samples_per_domain: int = 50) -> List[Dict]:
        """Create balanced stress test dataset"""
        print("\n" + "="*60)
        print("CREATING STRESS TEST DATASET")
        print("="*60)
        
        all_data = []
        
        # Process each adapter
        for adapter in self.adapters:
            data = adapter.process(max_samples=samples_per_domain)
            all_data.extend(data)
        
        # Add adversarial examples
        adversarial = generate_adversarial_examples()
        all_data.extend(adversarial)
        
        # Shuffle
        random.seed(42)
        random.shuffle(all_data)
        
        return all_data
    
    def save_dataset(self, data: List[Dict], output_path: str):
        """Save dataset as JSONL"""
        with open(output_path, 'w') as f:
            for item in data:
                serializable_item = to_json_serializable(item)
                f.write(json.dumps(serializable_item) + '\n')
        
        print(f"\n{'='*60}")
        print(f"SAVED STRESS TEST DATASET")
        print(f"{'='*60}")
        print(f"Location: {output_path}")
        print(f"Total samples: {len(data)}")
        
        # Print domain distribution
        domain_counts = Counter(item['domain'] for item in data)
        print(f"\n📊 Domain Distribution:")
        for domain in sorted(domain_counts.keys()):
            count = domain_counts[domain]
            pct = (count / len(data)) * 100
            print(f"  {domain:20s}: {count:5d} ({pct:5.1f}%)")


# ==================== MAIN ====================

def main():
    print("="*60)
    print("STRESS TEST DATASET CURATION")
    print("="*60)
    
    # Create mixer
    mixer = StressTestMixer()
    
    # Create dataset
    stress_test_data = mixer.create_stress_test_dataset(
        samples_per_domain=SAMPLES_PER_DOMAIN
    )
    
    # Save
    mixer.save_dataset(stress_test_data, OUTPUT_PATH)
    
    # Save sample for inspection
    sample_path = OUTPUT_PATH.replace('.jsonl', '_sample.jsonl')
    with open(sample_path, 'w') as f:
        for item in stress_test_data[:20]:
            f.write(json.dumps(to_json_serializable(item)) + '\n')
    
    print(f"\nSaved 20 samples to {sample_path}")
    print("\n✅ STRESS TEST DATASET CREATION COMPLETE!")
    
    return stress_test_data


In [17]:
main()

STRESS TEST DATASET CURATION

CREATING STRESS TEST DATASET

Processing GSM-Symbolic...
  [1/4] Loading raw data...
        Loaded 5000 rows
  [2/4] Transforming to unified schema...
        Transformed 5000 samples
  [3/4] Validating...
        Validated 5000 samples
  [4/4] Sampling 100 from 5000...

Processing HumanEvalPlus...
  [1/4] Loading raw data...
        Loaded 164 rows
  [2/4] Transforming to unified schema...
        Transformed 164 samples
  [3/4] Validating...
        Validated 164 samples
  [4/4] Sampling 100 from 164...

Processing SciQ...
  [1/4] Loading raw data...
        Loaded 13679 rows
  [2/4] Transforming to unified schema...
        Transformed 13679 samples
  [3/4] Validating...
        Validated 13670 samples
  [4/4] Sampling 100 from 13670...

Processing QMSum...
  [1/4] Loading raw data...
        Loaded 281 rows
  [2/4] Transforming to unified schema...
        Transformed 0 samples
  [3/4] Validating...
        Validated 0 samples

Processing LitBench-Rat

[{'domain': 'science',
  'prompt': 'Based on the following context, answer the question.\n\nContext: In the digestive system, food is digested and its nutrients are absorbed by the blood for transport around the body. The blood is part of the cardiovascular system.\n\nQuestion: Nutrients from food are absorbed by the blood for transport around the body as part of what system?',
  'answer': 'digestive',
  'metadata': {'source': 'SciQ'}},
 {'domain': 'science',
  'prompt': 'Based on the following context, answer the question.\n\nContext: We tend to think of solids as those materials that are solid at room temperature. However, all materials have melting points of some sort. Gases become solids at extremely low temperatures, and liquids will also become solid if the temperature is low enough. The Table below gives the melting points of some common materials.\n\nQuestion: When do gases and liquids become solids?',
  'answer': 'at low temperatures',
  'metadata': {'source': 'SciQ'}},
 {'dom